# CBIS-DDSM - CNN Model Training

This notebook trains a Convolutional Neural Network (CNN) on the preprocessed CBIS-DDSM mammogram images.

**Pipeline overview**:
- Load PNG images generated from the DICOM preprocessing step (benign / malignant folders)
- Create train/validation split
- Define a CNN (ResNet-18 fine-tuning)
- Train and evaluate the model
- Save the trained model and metadata into the `cbis_ddsm_dataset/models` folder

Before running, make sure:
- Your processed images are stored in a directory with subfolders `benign/` and `malignant/` (as created by the preprocessing notebook)
- You have installed `torch` and `torchvision` (CPU-only is fine) along with the existing project requirements.


In [1]:
# 1. Imports and configuration

import os
import random
from pathlib import Path

import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report

# Ensure reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")


Using device: cpu


In [2]:
# 2. Paths and hyperparameters

# TODO: update this path to where your processed CBIS-DDSM PNGs live
# It should contain subfolders 'benign' and 'malignant'
DATA_DIR = r"D:\\manifest-ZkhPvrLo5216730872708713142\\CBIS-DDSM_processed"

# Directory inside this repo where we'll save the trained model
# We assume you start Jupyter from the project root (DMML)
CBIS_ROOT = Path.cwd().resolve() / "cbis_ddsm_dataset"
MODELS_DIR = CBIS_ROOT / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)

BATCH_SIZE = 16
NUM_EPOCHS = 10
TRAIN_SPLIT = 0.8
LEARNING_RATE = 1e-4

print(f"DATA_DIR: {DATA_DIR}")
print(f"Models will be saved to: {MODELS_DIR}")


DATA_DIR: D:\\manifest-ZkhPvrLo5216730872708713142\\CBIS-DDSM_processed
Models will be saved to: C:\Users\aadi\Desktop\dmml\DMML-Group16\cbis_ddsm_dataset\model_training\cbis_ddsm_dataset\models


In [3]:
# 3. Dataset and DataLoaders

# Basic transforms: resize to 224x224, convert to tensor, normalize.
# We also convert to 3-channel grayscale so we can use ImageNet pre-trained models.

data_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# Create ImageFolder dataset (expects subfolders per class)
full_dataset = datasets.ImageFolder(DATA_DIR, transform=data_transforms)

num_samples = len(full_dataset)
num_train = int(TRAIN_SPLIT * num_samples)
num_val = num_samples - num_train

train_dataset, val_dataset = random_split(
    full_dataset,
    [num_train, num_val],
    generator=torch.Generator().manual_seed(SEED)
)

print(f"Total samples : {num_samples}")
print(f"Train samples : {len(train_dataset)}")
print(f"Val samples   : {len(val_dataset)}")
print(f"Classes       : {full_dataset.classes}")

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)


Total samples : 17846
Train samples : 14276
Val samples   : 3570
Classes       : ['benign', 'malignant']


In [ ]:
# 4. Define CNN model (ResNet-18 fine-tuning)

# Load a pre-trained ResNet-18 and replace the final layer for 2 classes
num_classes = 2

model = models.resnet18(pretrained=True)

# Freeze early layers if you want faster training / less overfitting
for param in model.parameters():
    param.requires_grad = True  # set False to freeze

in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)

model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print(model)


In [ ]:
# 5. Training loop

best_val_acc = 0.0
best_model_path = MODELS_DIR / "cbis_cnn_resnet18_best.pth"

for epoch in range(1, NUM_EPOCHS + 1):
    print(f"\nEpoch {epoch}/{NUM_EPOCHS}")
    print("-" * 30)

    # Training phase
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_train = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data).item()
        total_train += labels.size(0)

    epoch_train_loss = running_loss / total_train
    epoch_train_acc = running_corrects / total_train

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    val_running_corrects = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            val_running_loss += loss.item() * inputs.size(0)
            val_running_corrects += torch.sum(preds == labels.data).item()
            total_val += labels.size(0)

    epoch_val_loss = val_running_loss / total_val
    epoch_val_acc = val_running_corrects / total_val

    print(f"Train Loss: {epoch_train_loss:.4f}  Train Acc: {epoch_train_acc:.4f}")
    print(f"Val   Loss: {epoch_val_loss:.4f}  Val   Acc: {epoch_val_acc:.4f}")

    # Save best model
    if epoch_val_acc > best_val_acc:
        best_val_acc = epoch_val_acc
        torch.save(model.state_dict(), best_model_path)
        print(f"\n✓ New best model saved to: {best_model_path} (val_acc={best_val_acc:.4f})")


C:\Users\aadi\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Epoch 1/10
------------------------------


In [ ]:
# 6. Evaluation on validation set and save metadata

# Load best model weights
model.load_state_dict(torch.load(best_model_path, map_location=DEVICE))
model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

acc = accuracy_score(all_labels, all_preds)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="binary")

print("\n=== Validation Performance (Best Model) ===")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-score : {f1:.4f}")

print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=full_dataset.classes))

# Save a small metadata file alongside the model
import json

metadata = {
    "model_type": "ResNet18_CNN",
    "classes": full_dataset.classes,
    "num_epochs": NUM_EPOCHS,
    "train_split": TRAIN_SPLIT,
    "batch_size": BATCH_SIZE,
    "learning_rate": LEARNING_RATE,
    "val_accuracy": float(acc),
    "val_precision": float(precision),
    "val_recall": float(recall),
    "val_f1": float(f1),
}

metadata_path = MODELS_DIR / "cbis_cnn_resnet18_metadata.json"
with open(metadata_path, "w") as f:
    json.dump(metadata, f, indent=4)

print(f"\nMetadata saved to: {metadata_path}")
